# What this does

1. Clones the official repo → ensures we get the custom architecture (llava_mistral).

2. Installs dependencies from their requirements.txt (plus latest transformers).

3. Uses load_pretrained_model from the repo → avoids the error you got with AutoModel.

4. Runs a test query on a sample fundus image.

In [ ]:
# 1. Clone the official repo
!git clone https://github.com/microsoft/LLaVA-Med.git
%cd LLaVA-Med

Cloning into 'LLaVA-Med'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 451 (delta 54), reused 49 (delta 49), pack-reused 345 (from 2)
Receiving objects: 100% (451/451), 77.08 MiB | 20.76 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Filtering content: 100% (10/10), 813.63 MiB | 69.69 MiB/s, done.
/content/LLaVA-Med


In [ ]:
# 2. Install dependencies
!pip install -r requirements.txt
# !pip install git+https://github.com/huggingface/transformers.git
# !pip install git+https://github.com/huggingface/peft.git
# !pip install bitsandbytes accelerate pillow

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-b8jmr879
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-b8jmr879
  Resolved https://github.com/huggingface/transformers.git to commit 50d2448a1a7b75354c3d0ca879afd124abd244ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers

In [ ]:

# 3. Import libraries
import torch
from llava.model.builder import load_pretrained_model
from PIL import Image
import requests
import kagglehub

In [ ]:
model_path = "microsoft/llava-med-v1.5-mistral-7b"  # HuggingFace model repo

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name="llava-med-v1.5-mistral-7b"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/262M [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

TypeError: LlavaMistralForCausalLM.__init__() got an unexpected keyword argument 'use_flash_attention_2'

In [ ]:
kagglehub.dataset_download("waleedashraf9t/diabetic-retinopathy-fundus-images-resized")

100%|██████████| 14.6M/14.6M [00:00<00:00, 15.6MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/waleedashraf9t/diabetic-retinopathy-fundus-images-resized/versions/1'

In [ ]:
# 5. Test inference with a sample fundus image

url = "/label_0_1000.jpeg"

img = Image.open(url).convert("RGB")

question = "Does this image show signs of diabetic retinopathy?"

inputs = image_processor(images=img, text=question, return_tensors="pt").to(model.device)

with torch.inference_mode():
    output_ids = model.generate(**inputs, max_new_tokens=128)

answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Answer:", answer)

NameError: name 'image_processor' is not defined